$P$ is a priority set of objective functions, $p$ is probablity that the objective is added to $P$, $f_{rank} \in (0,1]$

### First Iteration

1. For each objective function $f_k \in P$:
    0. if $aspir_k$ is not set then $ref_n=icv_k$ 
    0. elif $rand$ < $p \cdot f_{rank}$ Set $ref_n=aspir_k$
2. set $iter=0$

### Following iterations
1. Set $p_p = p$
2. $iter$++
3. For each objective function $f_k$ ordered by priority:
    0. if $f_k(x)$ difference to $aspir$ is less than $tol$, add $f_k$ to set $P$ if $rand < p$.
    1. if difference to $aspir$ is greater than $tol$, add $f_k$ to set $P$ if $rand \cdot f_{rank} < p_p$.
       0. decrease $p_p$
3. If $p_p = p$ **STOP**
4. For each objective function $f_k \in P$:
    0. if $aspir_k$ is not set then $ref_n=icv_k$ 
    0. elif $rand$ < $p \cdot f_{rank}$ Set $ref_n=aspir_k-(aspir_k-f_k(x))/2$
5. For each objective function $f_k \not \in P$:
    0. set $ref_n = nadir_k$
6. if $iter \lt iter_{stop}$ *goto* step 2.
""")

# Reference point NSGAII

In [ ]:
from scipy import spatial
from scipy.optimize import differential_evolution
import numpy as np
import pyMOEA

def rNSGAII_solution(problem,refpoint,evals=10000,nf=3):
    points=pyMOEA.solve(pyMOEA.problem_def(problem,nf),refpoint=refpoint,evals=evals)
    A=np.array(points)
    return A[spatial.KDTree(A).query(refpoint)[1]]

def ACH(x,problem,ref,rho=0.001):
    f=pyMOEA.evaluate(problem,x)
    nadir=[1.0]*len(ref)
    utopian=[0.0]*len(ref)
    A=np.array([f,ref,nadir,utopian])    
    return np.max(np.apply_along_axis(lambda x:(x[0]-x[1])/(x[2]-x[3]),0,A)) \
                 +rho*np.sum(np.apply_along_axis(lambda x:x[0]/(x[2]-x[3]),0,A))
               
def ACH_solution(problem_name,refpoint,evals=10000,nf=3):
    problem=pyMOEA.problem(problem_name,nf)
    bounds=pyMOEA.bounds(problem)
    res=differential_evolution(ACH,bounds,args=(problem,[0.2,0.2,0.2]),maxiter=evals)
    return pyMOEA.evaluate(problem,res.x)
                               

def make_run(problem,ref,nf=3,runs=10):
    nsgaii=[]
    ach=[]
    best=[0,0]
    for f in range(runs):
        nsgaii.append(rNSGAII_solution(problem,ref))
        nd=spatial.distance.euclidean(ref,nsgaii[-1])
    
        ach.append(ACH_solution(problem,ref))
        ad=spatial.distance.euclidean(ref,ach[-1])
    
        if ad<nd:
            best[0]+=1
        
        else:
            best[1]+=1
        
    if best[0]>best[1]:
        print "ACH"
    else:
        preint "rNSGAII"
    return (best,ach,nsgaii)

In [ ]:
ref=[0.2,0.2,0.2]
res=[]
for i,obj in enumerate(["Two","Three","Four"]):
    print "%s objectives"%obj
    for method in ['DTLZ1','DTLZ2']:
        print method
        res.append(make_run(method,ref,i))
        if res[-][0][0] >  res[-][0][0]:
            print "ACH"
        else:
            print "rNSGAII"
    